In [185]:
import warnings
warnings.filterwarnings('ignore')

In [186]:
#Text pre-processing
"""removes punctuation, stopwords, and returns a list of the remaining words, or tokens"""
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VenuGopal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\VenuGopal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [187]:
# Cleaning the text
import string

def text_process(text):
    '''
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Return the cleaned text as a list of words
    4. Remove words
    '''
    stemmer = WordNetLemmatizer()
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join([i for i in nopunc if not i.isdigit()])
    nopunc =  [word.lower() for word in nopunc.split() if word not in stopwords.words('english')]
    
#   Print each chunk
#   print([stemmer.lemmatize(word) for word in nopunc]) 

    return [stemmer.lemmatize(word) for word in nopunc]

In [188]:
# importing libraries 
import speech_recognition as sr 
import os 
from pydub import AudioSegment
from pydub.silence import split_on_silence

# create a speech recognition object
r = sr.Recognizer()

# a function that splits the audio file into chunks
# and applies speech recognition
def get_large_audio_transcription(path):
    
    # Cleaned paragraph from given audio file
    cleaned_paragraph = []
    
    # Get audio file name from given path
    # print(path)
    audioFileName = path.split('/')[-1]
    
    # open the audio file using pydub
    sound = AudioSegment.from_file(path, "flac")    
    
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    
    # split audio sound where silence is 500 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
    
    # experiment with this value for your target audio file
        min_silence_len = 500,
    
    # adjust this per requirement
        silence_thresh = sound.dBFS-14,
    
    # keep the silence for 500 milli seconds, adjustable as well
        keep_silence=500,
    )
    
    folder_name = "audio-chunks"
    
    # create a directory to store the audio chunks
    
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
        
    whole_text = ""
    
#     # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        
        # export audio chunk and save it in
        # the `folder_name` directory.
        
        chunk_filename = os.path.join(folder_name, f"{audioFileName}-chunk-{i}.flac")
        audio_chunk.export(chunk_filename, format="flac")
        
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            
        # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}"
                
                
                '''
                # Print each chunk of audio file

                #  print(chunk_filename, ":", text)

                '''
                
                '''
                # Process the transcripted text - Cleaning

                #  cleaned_paragraph += text_process(text)

                '''            
                whole_text += text
    
    '''
    # Print Cleaned Paragraph 
        
    #  print("\nCleaned Paragraph :\n",cleaned_paragraph)
    
    '''
    
    # return the text for all chunks detected
    
    return whole_text

In [189]:
import os

# files_19_198 - List of audio file paths in folder ./19/198/
# files_19_227 - List of audio file paths in folder ./19/227/

files_19_198 = ['./19/198/' + f for f in os.listdir('./19/198/')[:-1]]
files_19_227 = ['./19/227/' + f for f in os.listdir('./19/227/')[:-1]]

# print(files_19_198)
# print(files_19_227)

result_transcription_19_198 = []
result_transcription_19_227 = []

for eachFilePath in files_19_198:
    res = ""
    wholeText = get_large_audio_transcription(eachFilePath)
    # print(eachFilePath.split('/')[-1]+" :\n", wholeText)
    result_transcription_19_198 += [res.join(wholeText)]
    
for eachFilePath in files_19_227:
    res = ""
    wholeText = get_large_audio_transcription(eachFilePath)
    # print(eachFilePath.split('/')[-1]+" :\n", wholeText)
    result_transcription_19_227 += [res.join(wholeText)]

Error: 
Error: 
Error: 


In [190]:
res_trans_19_198 = result_transcription_19_198
res_trans_19_227 = result_transcription_19_227

In [191]:
import pandas as pd
import numpy as np

In [192]:
result_transcription_19_198 = np.array(result_transcription_19_198).reshape(38, 1)
result_transcription_19_227 = np.array(result_transcription_19_227).reshape(73, 1)

df_res_19_198 = pd.DataFrame(result_transcription_19_198)
df_res_19_227 = pd.DataFrame(result_transcription_19_227)

df_res_19_198.columns = ['Model Transcription']
df_res_19_227.columns = ['Model Transcription']

In [193]:
trans_file_19_198 = pd.read_csv("./19/198/19-198.trans.txt", header = None)
trans_file_19_227 = pd.read_csv("./19/227/19-227.trans.txt", header = None)

In [194]:
res_file_19_198 = []
res_file_19_227 = []

for each_line in trans_file_19_198[0]:
    res_file_19_198 += [each_line.split(' ', 1)]

for each_line in trans_file_19_227[0]:
    res_file_19_227 += [each_line.split(' ', 1)]
    
df_19_198 = pd.DataFrame(res_file_19_198)
df_19_227 = pd.DataFrame(res_file_19_227)

df_19_198.columns = ['File Name', 'Original Transcription']
df_19_227.columns = ['File Name', 'Original Transcription']

In [195]:
final_res_19_198 = [df_19_198, df_res_19_198]
final_res_19_227 = [df_19_227, df_res_19_227]

final_res_19_198 = pd.concat(final_res_19_198, axis=1)
final_res_19_227 = pd.concat(final_res_19_227, axis=1)

In [196]:
final_res_19_198.head()

,File Name,Original Transcription,Model Transcription
0,19-198-0000,NORTHANGER ABBEY,Northanger abbey
1,19-198-0001,THIS LITTLE WORK WAS FINISHED IN THE YEAR EIGH...,This work was finished in the year 1830And int...
2,19-198-0002,NEITHER THE AUTHOR NOR THE PUBLIC HAVE ANY OTH...,Neither the awesome vinod public have any othe...
3,19-198-0003,THE PUBLIC ARE ENTREATED TO BEAR IN MIND THAT ...,Tatha black are interested to bear in mind tha...
4,19-198-0004,CHAPTER ONE NO ONE WHO HAD EVER SEEN CATHERINE...,Chapter 1No one had ever seen cancer in mobile...


In [197]:
final_res_19_227.head()

,File Name,Original Transcription,Model Transcription
0,19-227-0000,CHAPTER THIRTY CATHERINE'S DISPOSITION WAS NOT...,Chapter 30762 session was naturally 7:309 habi...
1,19-227-0001,HER MOTHER COULD NOT BUT PERCEIVE THEM NOW TO ...,Mother could not receive them now to be greatl...
2,19-227-0002,AND IT SEEMED AS IF SHE COULD EVEN WALK ABOUT ...,Seemed as if she could even walked about the h...
3,19-227-0003,BUT WHEN A THIRD NIGHT'S REST HAD NEITHER REST...,The night's rest of the restored hatya for thi...
4,19-227-0004,MY DEAR CATHERINE I AM AFRAID YOU ARE GROWING ...,My dear friendI am afraid you are growing quit...


In [198]:
final_res_19_198['Model Transcription'] = final_res_19_198['Model Transcription'].str.lower()
final_res_19_198['Original Transcription'] = final_res_19_198['Original Transcription'].str.lower() 

final_res_19_227['Model Transcription'] = final_res_19_227['Model Transcription'].str.lower()
final_res_19_227['Original Transcription'] = final_res_19_227['Original Transcription'].str.lower()

final_res_19_198.to_csv('./19/Result_19_198.csv')
final_res_19_227.to_csv('./19/Result_19_227.csv')

In [199]:
# PART-2, SEQUENCE MATCHING

In [200]:
from difflib import SequenceMatcher

In [201]:
modelTranscriptions_19_198 = []
originalTranscriptions_19_198 = []
sequenceSimilarity_19_198 = []

for model_transcription in final_res_19_198['Model Transcription']:
    modelTranscriptions_19_198.append(model_transcription)
    
for original_transcription in final_res_19_198['Original Transcription']:
    originalTranscriptions_19_198.append(original_transcription)

for i in range(len(modelTranscriptions_19_198)):
    sequenceSimilarity_19_198.append(round(SequenceMatcher(None, originalTranscriptions_19_198[i], modelTranscriptions_19_198[i]).ratio(), 3))

In [202]:
modelTranscriptions_19_227 = []
originalTranscriptions_19_227 = []
sequenceSimilarity_19_227 = []

for model_transcription in final_res_19_227['Model Transcription']:
    modelTranscriptions_19_227.append(model_transcription)
    
for original_transcription in final_res_19_227['Original Transcription']:
    originalTranscriptions_19_227.append(original_transcription)

for i in range(len(modelTranscriptions_19_227)):
    sequenceSimilarity_19_227.append(round(SequenceMatcher(None, originalTranscriptions_19_227[i], modelTranscriptions_19_227[i]).ratio(), 3))

In [203]:
sequenceSimilarity_19_198 = np.array(sequenceSimilarity_19_198).reshape(38, 1)
sequenceSimilarity_19_227 = np.array(sequenceSimilarity_19_227).reshape(73, 1)

In [204]:
df_similarity_19_198 = pd.DataFrame(sequenceSimilarity_19_198)
df_similarity_19_227 = pd.DataFrame(sequenceSimilarity_19_227)

df_similarity_19_198.columns = ['Sequence Similarity']
df_similarity_19_227.columns = ['Sequence Similarity']

In [205]:
Output_19_198 = [final_res_19_198, df_similarity_19_198]
Output_19_198 = pd.concat(Output_19_198, axis = 1)
Output_19_198.set_index('File Name', inplace = True)

Output_19_227 = [final_res_19_227, df_similarity_19_227]
Output_19_227 = pd.concat(Output_19_227, axis = 1)
Output_19_227.set_index('File Name', inplace = True)

In [206]:
Output_19_198.head()

,Original Transcription,Model Transcription,Sequence Similarity
File Name,,,
19-198-0000,northanger abbey,northanger abbey,1.000
19-198-0001,this little work was finished in the year eigh...,this work was finished in the year 1830and int...,0.835
19-198-0002,neither the author nor the public have any oth...,neither the awesome vinod public have any othe...,0.880
19-198-0003,the public are entreated to bear in mind that ...,tatha black are interested to bear in mind tha...,0.028
19-198-0004,chapter one no one who had ever seen catherine...,chapter 1no one had ever seen cancer in mobile...,0.798


In [207]:
Output_19_227.head()

,Original Transcription,Model Transcription,Sequence Similarity
File Name,,,
19-227-0000,chapter thirty catherine's disposition was not...,chapter 30762 session was naturally 7:309 habi...,0.632
19-227-0001,her mother could not but perceive them now to ...,mother could not receive them now to be greatl...,0.018
19-227-0002,and it seemed as if she could even walk about ...,seemed as if she could even walked about the h...,0.866
19-227-0003,but when a third night's rest had neither rest...,the night's rest of the restored hatya for thi...,0.765
19-227-0004,my dear catherine i am afraid you are growing ...,my dear friendi am afraid you are growing quit...,0.809


In [220]:
MSE_Training_DataSet = np.square(np.subtract(np.ones(sequenceSimilarity_19_198.shape, dtype=int),sequenceSimilarity_19_198)).mean()
MSE_Test_DataSet = np.square(np.subtract(np.ones(sequenceSimilarity_19_227.shape, dtype=int),sequenceSimilarity_19_227)).mean()

print('Mean Squared Error - Training Data Set : ', MSE_Training_DataSet)
print('Mean Squared Error - Test Data Set  : ', MSE_Test_DataSet)

Mean Squared Error - Training Data Set :  0.08906649999999999
Mean Squared Error - Test Data Set  :  0.11902001369863012
